In [24]:
import aiohttp
import asyncio
import json
async def fetch_country_slug(session, id):
    url = 'https://www.scholaro.com/gpa-calculator/Home/GetCountrySlug'
    headers = {
        'accept': 'application/json, text/javascript, */*; q=0.01',
        'accept-language': 'ko,en-US;q=0.9,en;q=0.8',
        'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'dnt': '1',
        'origin': 'https://www.scholaro.com',
        'priority': 'u=1, i',
        'referer': 'https://www.scholaro.com/gpa-calculator/',
        'sec-ch-ua': '"Chromium";v="133", "Not(A:Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors', 
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36',
        'x-requested-with': 'XMLHttpRequest'
    }
    data = {'id': str(id)}
    
    try:
        async with session.post(url, headers=headers, data=data) as response:
            if response.status == 200:
                return await response.json()
            else:
                print(f"Error for ID {id}: Status {response.status}")
                return None
    except Exception as e:
        print(f"Exception for ID {id}: {str(e)}")
        return None


async def fetch_scales(session, country_id, is_common):
    url = 'https://www.scholaro.com/gpa-calculator/Home/Scales_Read'
    headers = {
        'accept': '*/*',
        'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'origin': 'https://www.scholaro.com',
        'referer': 'https://www.scholaro.com/gpa-calculator/',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36',
        'x-requested-with': 'XMLHttpRequest'
    }
    
    data = {
        'countries': str(country_id),
        'scaleFilter': '',
        'common': '0' if is_common else '1',
        'orgId': '',
        'filter[filters][0][field]': 'Value',
        'filter[filters][0][operator]': 'eq', 
        'filter[filters][0][value]': str(country_id),
        'filter[logic]': 'and'
    }
    
    try:
        async with session.post(url, headers=headers, data=data) as response:
            if response.status == 200:
                return await response.json()
            else:
                print(f"Error for country ID {country_id}: Status {response.status}")
                return None
    except Exception as e:
        print(f"Exception for country ID {country_id}: {str(e)}")
        return None



async def get_all_country_slugs(country_ids):
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_country_slug(session, id) for id in country_ids]
        results = await asyncio.gather(*tasks)
        
        
        zip_results = zip(country_ids, results)
        # Filter out None results and create id-slug mapping
        
        return dict(zip_results)
    
    
async def get_all_scales(country_ids, is_common):
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_scales(session, id, is_common) for id in country_ids]
        results = await asyncio.gather(*tasks)
        
        # Create a dictionary mapping country IDs to their scale data
        scales_dict = {id: result for id, result in zip(country_ids, results) if result is not None}
        return scales_dict

# Get scales for countries found in slugs
country_ids = [i for i in range(1, 1001)]
common_scales = await get_all_scales(country_ids, True)
all_scales = await get_all_scales(country_ids, False)
# Run the async function
slugs = await get_all_country_slugs(country_ids)


data = {country_id: {'common_scales': common_scales.get(country_id, []), 'all_scales': all_scales.get(country_id, []), 'slug': slugs.get(country_id)} for country_id in set(common_scales.keys()) | set(all_scales.keys()) | set(slugs.keys())}

Error for ID 38: Status 500
Error for ID 122: Status 500
Error for ID 133: Status 500
Error for ID 136: Status 500
Error for ID 137: Status 500
Error for ID 140: Status 500
Error for ID 143: Status 500
Error for ID 148: Status 500
Error for ID 170: Status 500
Error for ID 167: Status 500
Error for ID 195: Status 500
Error for ID 198: Status 500
Error for ID 211: Status 500
Error for ID 202: Status 500
Error for ID 205: Status 500
Error for ID 238: Status 500
Error for ID 239: Status 500
Error for ID 241: Status 500
Error for ID 252: Status 500
Error for ID 254: Status 500
Error for ID 255: Status 500
Error for ID 257: Status 500
Error for ID 256: Status 500
Error for ID 259: Status 500
Error for ID 260: Status 500
Error for ID 262: Status 500
Error for ID 263: Status 500
Error for ID 264: Status 500
Error for ID 265: Status 500
Error for ID 266: Status 500
Error for ID 273: Status 500
Error for ID 268: Status 500
Error for ID 270: Status 500
Error for ID 267: Status 500
Error for ID 27

In [ ]:
# Get scales for countries found in slugs
country_ids = [i for i in range(1000, 2001)]
scales = await get_all_scales(country_ids)
# Run the async function
slugs = await get_all_country_slugs(country_ids)

augmented_Data = {country_id: {'scales': scales.get(country_id, []), 'slug': slugs.get(country_id)} for country_id in set(scales.keys()) | set(slugs.keys())}

In [ ]:
augmented_data2 = {k: v for k, v in augmented_Data.items() if v['slug']}
len(augmented_data2)

In [25]:
clean_data = {k: v for k, v in data.items() if v['slug']}
len(clean_data)

235

In [30]:
transformed_data = {}
for i in clean_data:
    transformed_data[clean_data[i]['slug']] = {
        'country_id': i,
        'common_scales': clean_data[i]['common_scales'],
        'all_scales': clean_data[i]['all_scales'],
    }
    

In [34]:
for i in transformed_data: 
    if 'korea' in i.lower() :
        print(i)
        print(transformed_data[i])
        break

South-Korea
{'country_id': 82, 'common_scales': [{'GradingScaleID': 97, 'CountryID': 0, 'GradingScaleName': 'Post-secondary', 'SortOrder': 10, 'WebEnabled': True, 'CreationDateTime': '0001-01-01T00:00:00', 'LastChangeDateTime': '0001-01-01T00:00:00', 'EducationStageID': 10}, {'GradingScaleID': 158, 'CountryID': 0, 'GradingScaleName': 'Secondary (October 2004–Present)', 'GradingScaleDescription': 'The rank is based on the relative position of a student in a class.', 'SortOrder': 20, 'WebEnabled': True, 'CreationDateTime': '0001-01-01T00:00:00', 'LastChangeDateTime': '0001-01-01T00:00:00', 'EducationStageID': 5}, {'GradingScaleID': 159, 'CountryID': 0, 'GradingScaleName': 'Secondary (Before October 2004)', 'SortOrder': 30, 'WebEnabled': True, 'CreationDateTime': '0001-01-01T00:00:00', 'LastChangeDateTime': '0001-01-01T00:00:00', 'EducationStageID': 5}, {'GradingScaleID': 1020, 'CountryID': 0, 'GradingScaleName': 'Letter Grade (Old)', 'SortOrder': 45, 'WebEnabled': True, 'Uncommon': False

In [54]:
import re


async def get_grading_scale(scale_id: int):
    """
    Get grading scale data from scholaro.com for a given scale ID
    Returns parsed grading scale data as a list of dictionaries
    """
    url = f'https://www.scholaro.com/gpa-calculator/Home/GradingScale?id={scale_id}'
    
    headers = {
        'accept': 'text/html, */*; q=0.01',
        'accept-language': 'ko,en-US;q=0.9,en;q=0.8',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors', 
        'sec-fetch-site': 'same-origin',
        'x-requested-with': 'XMLHttpRequest',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36'
    }
    
    async with aiohttp.ClientSession() as session:
        async with session.get(url, headers=headers) as response:
            html = await response.text()
            
            # New regex pattern to match the data structure
            data_match = re.search(r'"data":({.*?"Data":\[(.*?)\].*?})', html)
            if not data_match:
                return None
                
            try:
                data_str = data_match.group(1)
                # Clean up the data string to make it valid JSON
                data_str = data_str.replace('\\"', '"')
                data = json.loads(data_str)
                
                # Extract just the grading scale data
                grades = []
                for grade in data.get('Data', []):
                    grades.append({
                        'letter_grade': grade.get('LetterGrade', '').strip(),
                        'scale_from': grade.get('ScaleFrom'),
                        'scale_to': grade.get('ScaleTo'),
                        'description': grade.get('GradeDescriptionEnglish', ''),
                        'grade_points': grade.get('GradePoints'),
                        'notes': grade.get('Notes', '')
                    })
                    
                return grades
                
            except (json.JSONDecodeError, KeyError) as e:
                print(f"Error parsing data: {e}, scale_id: {scale_id}")
                return None




In [55]:
import asyncio

async def fetch_scale_data(common_scale):
    scale_data = await get_grading_scale(common_scale['GradingScaleID'])
    common_scale['scale_data'] = scale_data
    return common_scale

async def update_scales(item):
    common_scales = item['common_scales']
    all_scales = item['all_scales']

    # Use asyncio.gather to fetch scale data concurrently
    tasks_common = [fetch_scale_data(common_scale) for common_scale in common_scales]
    tasks_all = [fetch_scale_data(all_scale) for all_scale in all_scales]

    updated_common_scales = await asyncio.gather(*tasks_common)
    updated_all_scales = await asyncio.gather(*tasks_all)

    # Update the original common_scales with the fetched data
    item['common_scales'] = updated_common_scales
    item['all_scales'] = updated_all_scales
    return item

# Create a list to hold all update tasks
update_tasks = [update_scales(transformed_data[i]) for i in transformed_data]

# Execute all update tasks concurrently
updated_transformed_data = await asyncio.gather(*update_tasks)

# If you need to update the original transformed_data with the results:
for i, updated_item in zip(transformed_data.keys(), updated_transformed_data):
    transformed_data[i] = updated_item




Error parsing data: Expecting ',' delimiter: line 1 column 2155 (char 2154), scale_id: 313
Error parsing data: Expecting ',' delimiter: line 1 column 2155 (char 2154), scale_id: 313
Error parsing data: Expecting ',' delimiter: line 1 column 169 (char 168), scale_id: 1479


In [59]:
def extract_data(script_string: str) -> str:
    """Extracts the 'Data' part from a kendoGrid script string.

    Args:
        script_string: A string containing the kendoGrid script.

    Returns:
        A string containing the 'Data' part of the script, or None if not found.
    """
    start_index = script_string.find('"data":{"Data":[')
    if start_index == -1:
        return None
    start_index += len('"data":{"Data":[')
    end_index = script_string.find('],"Total":')
    if end_index == -1:
        return None
    return script_string[start_index:end_index]


data313 = """{ "columns": [{ "title": "Grade", "attributes": { "style": "text-align: center;" }, "headerAttributes": { "style":
"justify-content: center;" }, "exportable": true, "field": "LetterGrade", "filterable": { "checkAll": false },
"encoded": true }, { "title": "Scale", "attributes": { "style": "text-align: center;" }, "headerAttributes": { "style":
"justify-content: center;" }, "template": "# if (ScaleFrom != null) { ##= ScaleFrom ## if (ScaleTo != null) { # - #=
ScaleTo ## } } #", "exportable": true, "field": "ScaleFrom", "filterable": { "checkAll": false }, "encoded": true }, {
"title": "Description", "attributes": { "style": "text-align: center;" }, "headerAttributes": { "style":
"justify-content: center;" }, "template": "# if (GradeDescriptionEnglish != null \u0026\u0026 GradeDescriptionEnglish !=
\u0027\u0027) { ##= GradeDescriptionEnglish ## if (GradeDescription != null \u0026\u0026 GradeDescription !=
\u0027\u0027) { # (#= GradeDescription #) # } } #", "exportable": true, "field": "GradeDescriptionEnglish",
"filterable": { "checkAll": false }, "encoded": true }, { "title": "US Grade Points", "attributes": { "style":
"text-align: center;" }, "headerAttributes": { "style": "justify-content: center;" }, "exportable": true, "field":
"GradePoints", "filterable": { "checkAll": false }, "encoded": true }, { "title": "Notes", "attributes": { "style":
"text-align: center;" }, "headerAttributes": { "style": "justify-content: center;" }, "exportable": true, "field":
"Notes", "filterable": { "checkAll": false }, "encoded": true }], "scrollable": { "height": "auto" }, "dataSource": {
"type": (function () { if (kendo.data.transports['aspnetmvc-ajax']) { return 'aspnetmvc-ajax'; } else { throw new
Error('The kendo.aspnetmvc.min.js script is not included.'); } })(), "transport": { "read": { "url": "" }, "prefix": ""
}, "schema": { "data": "Data", "total": "Total", "errors": "Errors", "model": { "id": "GradeID", "fields": { "GradeID":
{ "type": "number" }, "LetterGrade": { "type": "string" }, "ScaleFrom": { "type": "number", "defaultValue": null },
"ScaleTo": { "type": "number", "defaultValue": null }, "Scale2From": { "type": "number", "defaultValue": null },
"Scale2To": { "type": "number", "defaultValue": null }, "GradeDescription": { "type": "string" },
"GradeDescriptionEnglish": { "type": "string" }, "Division": { "type": "string" }, "Notes": { "type": "string" },
"GoodUntil": { "type": "date", "defaultValue": null }, "SortOrder": { "type": "number", "defaultValue": null },
"USGradeID": { "type": "number", "defaultValue": null }, "USGrade": { "type": "string" }, "USGradeDesc": { "type":
"string" }, "GradePoints": { "type": "string" }, "GradePointsNZ": { "type": "string" }, "GradePointsIL": { "type":
"string" }, "GradePointsMR": { "type": "string" }, "GradePointsGL": { "type": "string" }, "CountryID": { "type":
"number", "defaultValue": null }, "EducationStageID": { "type": "number", "defaultValue": null }, "EducationStageName":
{ "type": "string" }, "GradingScaleID": { "type": "number" }, "CreationDateTime": { "type": "date" },
"LastChangeDateTime": { "type": "date" }, "CreationBy": { "type": "string" }, "LastChangeBy": { "type": "string" } } }
}, "data": { "Data": [{ "GradeID": 2193, "LetterGrade": "SS ", "ScaleFrom": 9.00, "ScaleTo": 10.00, "Scale2From": null,
"Scale2To": null, "GradeDescription": null, "GradeDescriptionEnglish": "Superior", "Division": "", "Notes": "",
"GoodUntil": null, "SortOrder": 10, "USGradeID": 98, "USGrade": null, "USGradeDesc": null, "GradePoints": "4.0",
"GradePointsNZ": null, "GradePointsIL": null, "GradePointsMR": null, "GradePointsGL": null, "CountryID": null,
"EducationStageID": null, "EducationStageName": null, "GradingScaleID": 313, "CreationDateTime":
"2011-01-27T21:52:18.41", "LastChangeDateTime": "2011-01-27T21:52:18.41", "CreationBy": "vizvek@gmail.com",
"LastChangeBy": "vizvek@gmail.com" }, { "GradeID": 2194, "LetterGrade": "MS ", "ScaleFrom": 7.00, "ScaleTo": 8.99,
"Scale2From": null, "Scale2To": null, "GradeDescription": "Médio Superior ", "GradeDescriptionEnglish": "Medium
Superior", "Division": "", "Notes": "", "GoodUntil": null, "SortOrder": 20, "USGradeID": 101, "USGrade": null,
"USGradeDesc": null, "GradePoints": "3.0", "GradePointsNZ": null, "GradePointsIL": null, "GradePointsMR": null,
"GradePointsGL": null, "CountryID": null, "EducationStageID": null, "EducationStageName": null, "GradingScaleID": 313,
"CreationDateTime": "2011-01-27T21:52:56.313", "LastChangeDateTime": "2011-01-27T21:55:56.577", "CreationBy":
"vizvek@gmail.com", "LastChangeBy": "vizvek@gmail.com" }, { "GradeID": 2195, "LetterGrade": "MM ", "ScaleFrom": 5.00,
"ScaleTo": 6.99, "Scale2From": null, "Scale2To": null, "GradeDescription": "Médio ", "GradeDescriptionEnglish":
"Medium", "Division": "", "Notes": "", "GoodUntil": null, "SortOrder": 25, "USGradeID": 104, "USGrade": null,
"USGradeDesc": null, "GradePoints": "2.0", "GradePointsNZ": null, "GradePointsIL": null, "GradePointsMR": null,
"GradePointsGL": null, "CountryID": null, "EducationStageID": null, "EducationStageName": null, "GradingScaleID": 313,
"CreationDateTime": "2011-01-27T21:53:46.847", "LastChangeDateTime": "2011-01-27T21:56:18.087", "CreationBy":
"vizvek@gmail.com", "LastChangeBy": "vizvek@gmail.com" }, { "GradeID": 2198, "LetterGrade": "CC ", "ScaleFrom": null,
"ScaleTo": null, "Scale2From": null, "Scale2To": null, "GradeDescription": "Crédito Concedido",
"GradeDescriptionEnglish": "Credit Granted", "Division": "", "Notes": "Represents course work completed at a standard of
\"C\" or above on the U.S. scale", "GoodUntil": null, "SortOrder": 28, "USGradeID": 2798, "USGrade": null,
"USGradeDesc": null, "GradePoints": "-", "GradePointsNZ": null, "GradePointsIL": null, "GradePointsMR": null,
"GradePointsGL": null, "CountryID": null, "EducationStageID": null, "EducationStageName": null, "GradingScaleID": 313,
"CreationDateTime": "2011-01-29T10:03:43.987", "LastChangeDateTime": "2011-01-29T10:03:43.987", "CreationBy":
"vizvek@gmail.com", "LastChangeBy": "vizvek@gmail.com" }, { "GradeID": 2196, "LetterGrade": "MI ", "ScaleFrom": 3.00,
"ScaleTo": 4.99, "Scale2From": null, "Scale2To": null, "GradeDescription": "Médio Inferior", "GradeDescriptionEnglish":
"Medium Inferior", "Division": "", "Notes": "Fail", "GoodUntil": null, "SortOrder": 30, "USGradeID": 109, "USGrade":
null, "USGradeDesc": null, "GradePoints": "0.0", "GradePointsNZ": null, "GradePointsIL": null, "GradePointsMR": null,
"GradePointsGL": null, "CountryID": null, "EducationStageID": null, "EducationStageName": null, "GradingScaleID": 313,
"CreationDateTime": "2011-01-27T21:55:32.167", "LastChangeDateTime": "2011-01-27T21:55:32.167", "CreationBy":
"vizvek@gmail.com", "LastChangeBy": "vizvek@gmail.com" }, { "GradeID": 2197, "LetterGrade": "II ", "ScaleFrom": 0.00,
"ScaleTo": 2.99, "Scale2From": null, "Scale2To": null, "GradeDescription": null, "GradeDescriptionEnglish": "Inferior",
"Division": "", "Notes": "Fail", "GoodUntil": null, "SortOrder": 40, "USGradeID": 109, "USGrade": null, "USGradeDesc":
null, "GradePoints": "0.0", "GradePointsNZ": null, "GradePointsIL": null, "GradePointsMR": null, "GradePointsGL": null,
"CountryID": null, "EducationStageID": null, "EducationStageName": null, "GradingScaleID": 313, "CreationDateTime":
"2011-01-27T21:58:10.77", "LastChangeDateTime": "2011-01-27T21:58:26.98", "CreationBy": "vizvek@gmail.com",
"LastChangeBy": "vizvek@gmail.com" }, { "GradeID": 2199, "LetterGrade": "SR ", "ScaleFrom": null, "ScaleTo": null,
"Scale2From": null, "Scale2To": null, "GradeDescription": "Sem Rendimento", "GradeDescriptionEnglish": "Fail",
"Division": "", "Notes": "Fail or no grade", "GoodUntil": null, "SortOrder": 50, "USGradeID": 2997, "USGrade": null,
"USGradeDesc": null, "GradePoints": "-", "GradePointsNZ": null, "GradePointsIL": null, "GradePointsMR": null,
"GradePointsGL": null, "CountryID": null, "EducationStageID": null, "EducationStageName": null, "GradingScaleID": 313,
"CreationDateTime": "2011-01-29T10:07:01.377", "LastChangeDateTime": "2011-01-29T10:07:01.377", "CreationBy":
"vizvek@gmail.com", "LastChangeBy": "vizvek@gmail.com" }], "Total": 7 }"""

data1479 = """<script>kendo.syncReady(function(){jQuery("#GradingScaleGrid").kendoGrid({"columns":[{"title":"Scale","attributes":{"style":"text-align: center;"},"headerAttributes":{"style":"justify-content: center;"},"template":"# if (ScaleFrom != null) { ##= ScaleFrom ## if (ScaleTo != null) { # - #= ScaleTo ## } } #","exportable":true,"field":"ScaleFrom","filterable":{"checkAll":false},"encoded":true},{"title":"Description","attributes":{"style":"text-align: center;"},"headerAttributes":{"style":"justify-content: center;"},"template":"# if (GradeDescriptionEnglish != null \u0026\u0026 GradeDescriptionEnglish != \u0027\u0027) { ##= GradeDescriptionEnglish ## if (GradeDescription != null \u0026\u0026 GradeDescription != \u0027\u0027) { # (#= GradeDescription #) # } } #","exportable":true,"field":"GradeDescriptionEnglish","filterable":{"checkAll":false},"encoded":true},{"title":"US Grade Points","attributes":{"style":"text-align: center;"},"headerAttributes":{"style":"justify-content: center;"},"exportable":true,"field":"GradePoints","filterable":{"checkAll":false},"encoded":true}],"scrollable":{"height":"auto"},"dataSource":{"type":(function(){if(kendo.data.transports['aspnetmvc-ajax']){return 'aspnetmvc-ajax';} else{throw new Error('The kendo.aspnetmvc.min.js script is not included.');}})(),"transport":{"read":{"url":""},"prefix":""},"schema":{"data":"Data","total":"Total","errors":"Errors","model":{"id":"GradeID","fields":{"GradeID":{"type":"number"},"LetterGrade":{"type":"string"},"ScaleFrom":{"type":"number","defaultValue":null},"ScaleTo":{"type":"number","defaultValue":null},"Scale2From":{"type":"number","defaultValue":null},"Scale2To":{"type":"number","defaultValue":null},"GradeDescription":{"type":"string"},"GradeDescriptionEnglish":{"type":"string"},"Division":{"type":"string"},"Notes":{"type":"string"},"GoodUntil":{"type":"date","defaultValue":null},"SortOrder":{"type":"number","defaultValue":null},"USGradeID":{"type":"number","defaultValue":null},"USGrade":{"type":"string"},"USGradeDesc":{"type":"string"},"GradePoints":{"type":"string"},"GradePointsNZ":{"type":"string"},"GradePointsIL":{"type":"string"},"GradePointsMR":{"type":"string"},"GradePointsGL":{"type":"string"},"CountryID":{"type":"number","defaultValue":null},"EducationStageID":{"type":"number","defaultValue":null},"EducationStageName":{"type":"string"},"GradingScaleID":{"type":"number"},"CreationDateTime":{"type":"date"},"LastChangeDateTime":{"type":"date"},"CreationBy":{"type":"string"},"LastChangeBy":{"type":"string"}}}},"data":{"Data":[{"GradeID":11547,"LetterGrade":"    ","ScaleFrom":85.00,"ScaleTo":100.00,"Scale2From":null,"Scale2To":null,"GradeDescription":null,"GradeDescriptionEnglish":"\"Distinction\"","Division":"","Notes":"","GoodUntil":null,"SortOrder":10,"USGradeID":98,"USGrade":null,"USGradeDesc":null,"GradePoints":"4.0","GradePointsNZ":null,"GradePointsIL":null,"GradePointsMR":null,"GradePointsGL":null,"CountryID":null,"EducationStageID":null,"EducationStageName":null,"GradingScaleID":1479,"CreationDateTime":"2022-10-05T17:17:34.923","LastChangeDateTime":"2022-10-05T17:17:34.923","CreationBy":"adonin@foreigncredits.com","LastChangeBy":"adonin@foreigncredits.com"},{"GradeID":11548,"LetterGrade":"    ","ScaleFrom":71.00,"ScaleTo":84.00,"Scale2From":null,"Scale2To":null,"GradeDescription":null,"GradeDescriptionEnglish":"\"Merit\"","Division":"","Notes":"","GoodUntil":null,"SortOrder":20,"USGradeID":99,"USGrade":null,"USGradeDesc":null,"GradePoints":"3.7","GradePointsNZ":null,"GradePointsIL":null,"GradePointsMR":null,"GradePointsGL":null,"CountryID":null,"EducationStageID":null,"EducationStageName":null,"GradingScaleID":1479,"CreationDateTime":"2022-10-05T17:17:51.89","LastChangeDateTime":"2022-10-05T17:17:51.89","CreationBy":"adonin@foreigncredits.com","LastChangeBy":"adonin@foreigncredits.com"},{"GradeID":11549,"LetterGrade":"    ","ScaleFrom":61.00,"ScaleTo":70.00,"Scale2From":null,"Scale2To":null,"GradeDescription":null,"GradeDescriptionEnglish":"\"Good\"","Division":"","Notes":"","GoodUntil":null,"SortOrder":30,"USGradeID":100,"USGrade":null,"USGradeDesc":null,"GradePoints":"3.3","GradePointsNZ":null,"GradePointsIL":null,"GradePointsMR":null,"GradePointsGL":null,"CountryID":null,"EducationStageID":null,"EducationStageName":null,"GradingScaleID":1479,"CreationDateTime":"2022-10-05T17:18:05.48","LastChangeDateTime":"2022-10-05T17:18:05.48","CreationBy":"adonin@foreigncredits.com","LastChangeBy":"adonin@foreigncredits.com"},{"GradeID":11550,"LetterGrade":"    ","ScaleFrom":50.00,"ScaleTo":60.00,"Scale2From":null,"Scale2To":null,"GradeDescription":null,"GradeDescriptionEnglish":"\"Pass\"","Division":"","Notes":"","GoodUntil":null,"SortOrder":40,"USGradeID":101,"USGrade":null,"USGradeDesc":null,"GradePoints":"3.0","GradePointsNZ":null,"GradePointsIL":null,"GradePointsMR":null,"GradePointsGL":null,"CountryID":null,"EducationStageID":null,"EducationStageName":null,"GradingScaleID":1479,"CreationDateTime":"2022-10-05T17:18:22.377","LastChangeDateTime":"2022-10-05T17:18:22.377","CreationBy":"adonin@foreigncredits.com","LastChangeBy":"adonin@foreigncredits.com"},{"GradeID":11552,"LetterGrade":"    ","ScaleFrom":null,"ScaleTo":null,"Scale2From":null,"Scale2To":null,"GradeDescription":null,"GradeDescriptionEnglish":"Pass","Division":"","Notes":"","GoodUntil":null,"SortOrder":45,"USGradeID":2798,"USGrade":null,"USGradeDesc":null,"GradePoints":"-","GradePointsNZ":null,"GradePointsIL":null,"GradePointsMR":null,"GradePointsGL":null,"CountryID":null,"EducationStageID":null,"EducationStageName":null,"GradingScaleID":1479,"CreationDateTime":"2022-10-05T17:30:22.65","LastChangeDateTime":"2022-10-05T17:30:22.65","CreationBy":"adonin@foreigncredits.com","LastChangeBy":"adonin@foreigncredits.com"},{"GradeID":11551,"LetterGrade":"    ","ScaleFrom":0.00,"ScaleTo":49.00,"Scale2From":null,"Scale2To":null,"GradeDescription":null,"GradeDescriptionEnglish":"Fail","Division":"","Notes":"","GoodUntil":null,"SortOrder":50,"USGradeID":109,"USGrade":null,"USGradeDesc":null,"GradePoints":"0.0","GradePointsNZ":null,"GradePointsIL":null,"GradePointsMR":null,"GradePointsGL":null,"CountryID":null,"EducationStageID":null,"EducationStageName":null,"GradingScaleID":1479,"CreationDateTime":"2022-10-05T17:21:28.613","LastChangeDateTime":"2022-10-05T17:21:28.613","CreationBy":"adonin@foreigncredits.com","LastChangeBy":"adonin@foreigncredits.com"}],"Total":6}}});});</script>"""


clean_data_1479 = extract_data(data1479)
clean_data_313 = extract_data(data313)

In [49]:
for i in transformed_data:
    if 'korea' in i.lower():
        print(i)

        print(transformed_data[i]['all_scales'])

South-Korea
[{'GradingScaleID': 2562, 'CountryID': 0, 'GradingScaleName': 'Chungbuk National University', 'WebEnabled': True, 'Uncommon': True, 'ReportsEnabled': True, 'CreationDateTime': '0001-01-01T00:00:00', 'LastChangeDateTime': '0001-01-01T00:00:00', 'scale_data': [{'letter_grade': 'A+', 'scale_from': 4.5, 'scale_to': None, 'description': '', 'grade_points': '4.0', 'notes': ''}, {'letter_grade': 'A0', 'scale_from': 4.0, 'scale_to': None, 'description': '', 'grade_points': '4.0', 'notes': ''}, {'letter_grade': 'B+', 'scale_from': 3.5, 'scale_to': None, 'description': '', 'grade_points': '3.5', 'notes': ''}, {'letter_grade': 'B0', 'scale_from': 3.0, 'scale_to': None, 'description': '', 'grade_points': '3.0', 'notes': ''}, {'letter_grade': 'C+', 'scale_from': 2.5, 'scale_to': None, 'description': '', 'grade_points': '2.5', 'notes': ''}, {'letter_grade': 'C0', 'scale_from': 2.0, 'scale_to': None, 'description': '', 'grade_points': '2.0', 'notes': ''}, {'letter_grade': 'D+', 'scale_fro

In [67]:
import json

keys_to_delete = []
for i, data in transformed_data.items():
    if not data['common_scales'] and not data['all_scales']:
        print(i)
        keys_to_delete.append(i)

for key in keys_to_delete:
    del transformed_data[key]

with open('transformed_data.json', 'w') as f:
    json.dump(transformed_data, f)


Canada
Falkland-Islands
French-Polynesia
Mayotte
Pitcairn-Islands
Solomon-Islands
St-Helena
St-Pierre-and-Miquelon
Wallis-and-Futuna-Islands
Saba
Belgium-French-Community
Guinea-Bissau
Czechoslovakia
Marshall-Islands
New-Caledonia
Palau
Comoros
San-Marino
Seychelles
East-Timor
Guam
